In [ ]:
import requests
import requests.auth
import pandas as pd
import re
import json

In [ ]:
def generate_header():
    f = json.load(open('C:/Users/elima/Desktop/Projects/Hinge Project/keys'))

    client_ID = f['reddit_api_key1']['client_ID']
    secret_key = f['reddit_api_key1']['secret_key']

    auth = requests.auth.HTTPBasicAuth(client_ID, secret_key)
    data = {
        'grant_type': 'password',
        'username': f['reddit_login']['user_name'],
        'password': f['reddit_login']['password']
    }

    headers = {'User-Agent': 'api_ver_1'}
    
    res = requests.post('https://www.reddit.com/api/v1/access_token',
                        auth=auth, data=data, headers=headers)
    res.json()

    access_token = res.json()['access_token']
    headers['Authorization'] = 'bearer ' + access_token
    
    return(headers)

In [ ]:
base_url = 'https://oauth.reddit.com'

hot_posts_payload = {'limit':100}
hot_posts = requests.get(base_url + '/r/hingeapp/hot', headers=generate_header(), params=hot_posts_payload).json()

In [ ]:
list_of_posts = []
reddit_url = 'https://www.reddit.com'
children_list = hot_posts['data']['children']

for post in children_list:
    if (post['data']['link_flair_richtext'][0]['t'] == 'Profile Review'):
        post_dic = {}
        post_title = post['data']['title']; post_dic['post title'] = post_title
        post_text = post['data']['selftext'][:250]; post_dic['post text'] = post_text
        found_age = re.search('[0-9][0-9]', post_title)
        if (found_age):
            age = found_age.group()
        else:
            age = None; 
        post_dic['age'] = age
        num_of_comments = post['data']['num_comments'] - 1; post_dic['number of comments'] = num_of_comments 
        num_of_ups = post['data']['ups']; post_dic['number of ups'] = num_of_ups
        post_url = post['data']['permalink'][21:28]; post_dic['post id'] = post_url
        list_of_posts.append(post_dic)
    
df = pd.DataFrame(list_of_posts)
pd.set_option('display.max_colwidth', None)
#pd.reset_option('all')

In [ ]:
df.to_csv('C:/Users/elima/Desktop/Projects/Hinge Project/HingeDataFramePosts')